In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

# Import our input dataset
champion_df = pd.read_csv("https://raw.githubusercontent.com/frlinh/game-analysis/main/Resources/csv/championMasteryBeta.csv")
champion_df.head()

,Date,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,8/27/2022,350,7,136999,1661483549000,115399,0,True,0,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb
1,8/27/2022,117,7,113568,1661644216000,91968,0,True,0,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb
2,8/27/2022,143,7,100598,1660524764000,78998,0,True,0,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb
3,8/27/2022,25,6,90688,1660354753000,69088,0,False,0,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb
4,8/27/2022,147,6,85820,1660539194000,64220,0,True,1,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb


In [2]:
# Total number of unique champions
champion_df["championId"].nunique()

161

In [3]:
# Drop all null values
cleaned_champion_df = champion_df.dropna()

In [4]:
# Drop columns
cleaned_champion_df = cleaned_champion_df.drop(columns=['Date', 'summonerId', 'lastPlayTime', 'championPointsSinceLastLevel', 'championPointsUntilNextLevel', 'tokensEarned'])
cleaned_champion_df

,championId,championLevel,championPoints,chestGranted
0,350,7,136999,True
1,117,7,113568,True
2,143,7,100598,True
3,25,6,90688,False
4,147,6,85820,True
...,...,...,...,...
8792,57,1,193,False
8793,200,1,171,False
8794,268,1,164,False
8795,37,1,154,False


In [5]:
# Show data types
cleaned_champion_df.dtypes

championId        int64
championLevel     int64
championPoints    int64
chestGranted       bool
dtype: object

In [7]:
# Remove difficulty outcome target from features data
y = cleaned_champion_df.championLevel
X = cleaned_champion_df.drop(columns="championLevel")

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [8]:
# Preprocess numerical data for neural network

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=10000)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.867


In [14]:
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu", input_dim=3))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
207/207 [==============================] - 1s 2ms/step - loss: -0.0015 - accuracy: 0.3267
Epoch 2/50
207/207 [==============================] - 0s 2ms/step - loss: -1.5057 - accuracy: 0.3524
Epoch 3/50
207/207 [==============================] - 0s 2ms/step - loss: -3.1714 - accuracy: 0.3524
Epoch 4/50
207/207 [==============================] - 0s 2ms/step - loss: -5.2710 - accuracy: 0.3524
Epoch 5/50
207/207 [==============================] - 0s 2ms/step - loss: -7.9242 - accuracy: 0.3524
Epoch 6/50
207/207 [==============================] - 0s 2ms/step - loss: -11.1326 - accuracy: 0.3524
Epoch 7/50
207/207 [==============================] - 0s 2ms/step - loss: -14.8609 - accuracy: 0.3524
Epoch 8/50
207/207 [==============================] - 0s 2ms/step - loss: -19.0657 - accuracy: 0.3524
Epoch 9/50
207/207 [==============================] - 0s 2ms/step - loss: -23.7204 - accuracy: 0.3524
Epoch 10/50
207/207 [==============================] - 0s 2ms/step - loss: -28.8050 - a